In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import numpy as np

# Time Series Dataset Class
class Time_Series_Dataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Split Series Function
def split_series(series, input_size, output_size, train_ratio, seed):
    np.random.seed(seed)
    data_length = len(series)
    train_size = int(data_length * train_ratio)
    
    X = []
    y = []
    
    for i in range(data_length - input_size - output_size):
        X.append(series[i:i+input_size])
        y.append(series[i+input_size:i+input_size+output_size])
    
    X = np.array(X)
    y = np.array(y)
    
    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)
    
    train_indices = indices[:train_size]
    test_indices = indices[train_size:]
    
    X_train = X[train_indices]
    y_train = y[train_indices]
    X_test = X[test_indices]
    y_test = y[test_indices]
    
    return X_train, X_test, y_train, y_test

# Convolutional LSTM Model
class ConvLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_filters, kernel_size):
        super(ConvLSTM, self).__init__()
        self.hidden_size = hidden_size

        # Convolutional layer
        self.conv1 = nn.Conv1d(in_channels=input_size, out_channels=num_filters, kernel_size=kernel_size, padding='same')
        
        # LSTM layer
        self.lstm = nn.LSTM(num_filters, hidden_size, batch_first=True, bidirectional=True)
        
        # Fully connected layer
        self.fc = nn.Linear(hidden_size * 2, output_size)

    def forward(self, x):
        # Convolutional layer expects input of shape (batch_size, in_channels, seq_length)
        x = x.permute(0, 2, 1)
        x = torch.relu(self.conv1(x))
        x = x.permute(0, 2, 1)  # Convert back to (batch_size, seq_length, num_filters)
        
        # LSTM layer
        out, _ = self.lstm(x)
        
        # Fully connected layer
        out = self.fc(out[:, -1, :])
        return out

# Load and preprocess the data
Bitcoin = pd.read_csv('data/coin_Ethereum.csv')
Close_Price = Bitcoin['Close'].copy()  # Assuming the Close price is the column named 'Close'
Close_Price_reshaped = np.array(Close_Price).reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
Close_Price_scaled = scaler.fit_transform(Close_Price_reshaped).flatten()

input_size = 1       
output_size = 5       
train_ratio = 0.8
seed = 5925
num_experiments = 30

rmse_train, mae_train, mape_train = [], [], []
rmse_train_steps = [[] for _ in range(output_size)]
mae_train_steps = [[] for _ in range(output_size)]
mape_train_steps = [[] for _ in range(output_size)]

rmse, mae, mape = [], [], []
rmse_steps = [[] for _ in range(output_size)]
mae_steps = [[] for _ in range(output_size)]
mape_steps = [[] for _ in range(output_size)]

for exp in range(num_experiments):
    X_train, X_test, y_train, y_test = split_series(Close_Price_scaled, input_size, output_size, train_ratio, seed)
    train_dataset = Time_Series_Dataset(X_train, y_train)
    test_dataset = Time_Series_Dataset(X_test, y_test)
    train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=False)
    test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

    hidden_size = 20  # Number of neurons for LSTM layer
    num_filters = 64  # Number of filters for Conv1D layer
    kernel_size = 2  # Kernel size for Conv1D layer

    model = ConvLSTM(input_size, hidden_size, output_size, num_filters, kernel_size)

    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    
    # Training loop
    num_epochs = 100 
    for epoch in range(1, num_epochs + 1):
        model.train()
        for inputs, targets in train_dataloader:
            inputs = inputs.unsqueeze(-1)  # Add feature dimension
            targets = targets
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    
    # Evaluate the model on the training set
    model.eval()
    y_train_pred = []
    y_train_actual = []
    
    with torch.no_grad():
        for inputs, targets in train_dataloader:
            inputs = inputs.unsqueeze(-1)
            targets = targets
            outputs = model(inputs)
            y_train_pred.append(outputs.numpy())
            y_train_actual.append(targets.numpy())
    
    # Convert lists to numpy arrays
    y_train_pred = np.concatenate(y_train_pred, axis=0)
    y_train_actual = np.concatenate(y_train_actual, axis=0)

    mse_train = mean_squared_error(y_train_actual, y_train_pred)
    rmse_train.append(np.sqrt(mse_train))
    
    for step in range(output_size):
        mse_train_step = mean_squared_error(y_train_pred[:, step], y_train_actual[:, step])
        rmse_train_steps[step].append(np.sqrt(mse_train_step))

    # Inverse Transform
    predicted_train_values = scaler.inverse_transform(y_train_pred)
    actual_train_values = scaler.inverse_transform(y_train_actual)
    
    mae_train.append(mean_absolute_error(actual_train_values, predicted_train_values))
    mape_train.append(mean_absolute_percentage_error(actual_train_values, predicted_train_values))
    
    actual_train_values_steps = list(zip(*actual_train_values))
    predicted_train_values_steps = list(zip(*predicted_train_values))
    
    for step in range(output_size):
        mae_train_steps[step].append(mean_absolute_error(actual_train_values_steps[step], predicted_train_values_steps[step]))
        mape_train_steps[step].append(mean_absolute_percentage_error(actual_train_values_steps[step], predicted_train_values_steps[step]))

    # Evaluate the model on the test set
    y_pred = []
    y_test_actual = []
    
    with torch.no_grad():
        for inputs, targets in test_dataloader:
            inputs = inputs.unsqueeze(-1)
            targets = targets
            outputs = model(inputs)
            y_pred.append(outputs.numpy())
            y_test_actual.append(targets.numpy())
    
    # Convert lists to numpy arrays
    y_pred = np.concatenate(y_pred, axis=0)
    y_test_actual = np.concatenate(y_test_actual, axis=0)

    mse = mean_squared_error(y_test_actual, y_pred)
    rmse.append(np.sqrt(mse))
    
    for step in range(output_size):
        mse_step = mean_squared_error(y_pred[:, step], y_test_actual[:, step])
        rmse_steps[step].append(np.sqrt(mse_step))

    # Inverse Transform
    predicted_values = scaler.inverse_transform(y_pred)
    actual_values = scaler.inverse_transform(y_test_actual)
    
    mae.append(mean_absolute_error(actual_values, predicted_values))
    mape.append(mean_absolute_percentage_error(actual_values, predicted_values))
    
    actual_values_steps = list(zip(*actual_values))
    predicted_values_steps = list(zip(*predicted_values))
    
    for step in range(output_size):
        mae_steps[step].append(mean_absolute_error(actual_values_steps[step], predicted_values_steps[step]))
        mape_steps[step].append(mean_absolute_percentage_error(actual_values_steps[step], predicted_values_steps[step]))

    print(f"Experiment {exp+1}/{num_experiments} done")
    seed += 1

print(f"Univariate Conv-LSTM Regression: After {num_experiments} experimental runs, here are the results:")
print(f"Across {output_size} predictive time steps, " +
      f"Avg RMSE: {np.mean(rmse):.4f} ± {np.std(rmse):.4f}, " +
      f"Avg MAE: {np.mean(mae):.2f} ± {np.std(mae):.2f}, " +
      f"Avg MAPE: {np.mean(mape)*100:.3f}% ± {np.std(mape)*100:.3f}%")
for step in range(output_size):
    print(
        f"At time step {step + 1}, "
        f"Avg RMSE: {np.mean(rmse_steps[step]):.4f} ± {np.std(rmse_steps[step]):.4f}, "
        f"Avg MAE: {np.mean(mae_steps[step]):.2f} ± {np.std(mae_steps[step]):.2f}, "
        f"Avg MAPE: {np.mean(mape_steps[step]) * 100:.3f}% ± {np.std(mape_steps[step]) * 100:.3f}%"
    )

print(f"Univariate Conv-LSTM Regression on Training Data: After {num_experiments} experimental runs, here are the results:")
print(f"Across {output_size} predictive time steps, " +
      f"Avg RMSE: {np.mean(rmse_train):.4f} ± {np.std(rmse_train):.4f}, " +
      f"Avg MAE: {np.mean(mae_train):.2f} ± {np.std(mae_train):.2f}, " +
      f"Avg MAPE: {np.mean(mape_train)*100:.3f}% ± {np.std(mape_train)*100:.3f}%")
for step in range(output_size):
    print(
        f"At time step {step + 1}, "
        f"Avg RMSE: {np.mean(rmse_train_steps[step]):.4f} ± {np.std(rmse_train_steps[step]):.4f}, "
        f"Avg MAE: {np.mean(mae_train_steps[step]):.2f} ± {np.std(mae_train_steps[step]):.2f}, "
        f"Avg MAPE: {np.mean(mape_train_steps[step]) * 100:.3f}% ± {np.std(mape_train_steps[step]) * 100:.3f}%"
    )


/Users/smruthirangarajan/Library/Python/3.9/lib/python/site-packages/torch/nn/modules/conv.py:306: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/Convolution.cpp:1032.)
  return F.conv1d(input, weight, bias, self.stride,


Experiment 1/30 done
Experiment 2/30 done
Experiment 3/30 done
Experiment 4/30 done
Experiment 5/30 done
Experiment 6/30 done
Experiment 7/30 done
Experiment 8/30 done
Experiment 9/30 done
Experiment 10/30 done
Experiment 11/30 done
Experiment 12/30 done
Experiment 13/30 done
Experiment 14/30 done
Experiment 15/30 done
Experiment 16/30 done
Experiment 17/30 done
Experiment 18/30 done
Experiment 19/30 done
Experiment 20/30 done
Experiment 21/30 done
Experiment 22/30 done
Experiment 23/30 done
Experiment 24/30 done
Experiment 25/30 done
Experiment 26/30 done
Experiment 27/30 done
Experiment 28/30 done
Experiment 29/30 done
Experiment 30/30 done
Univariate Conv-LSTM Regression: After 30 experimental runs, here are the results:
Across 5 predictive time steps, Avg RMSE: 0.0192 ± 0.0024, Avg MAE: 32.60 ± 3.85, Avg MAPE: 87.180% ± 56.631%
At time step 1, Avg RMSE: 0.0123 ± 0.0024, Avg MAE: 21.12 ± 3.97, Avg MAPE: 80.968% ± 61.538%
At time step 2, Avg RMSE: 0.0154 ± 0.0022, Avg MAE: 27.20 ± 3.

In [2]:
pd.DataFrame(actual_values).to_csv('ethereum_uni_clstm_classic_actual.csv')
pd.DataFrame(predicted_values).to_csv('ethereum_uni_clstm_classic_pred.csv')
pd.DataFrame(rmse_steps).transpose().to_csv('ethereum_uni_clstm_classic_rmse.csv')
pd.DataFrame(mae_steps).transpose().to_csv('ethereum_uni_clstm_classic_mae.csv')
pd.DataFrame(mape_steps).transpose().to_csv('ethereum_uni_clstm_classic_mape.csv')